<a href="https://colab.research.google.com/github/Gresey/Fake_News_Detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

In [23]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
fakeNews=pd.read_csv("/content/drive/MyDrive/News _dataset/Fake.csv")
trueNews=pd.read_csv("/content/drive/MyDrive/News _dataset/True.csv")

In [25]:
fakeNews.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [26]:
trueNews.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [42]:
fakeNews["class"] = 0
trueNews["class"] = 1

In [43]:
# shape of both the dataset
trueNews.shape, fakeNews.shape

((21407, 5), (23471, 5))

In [44]:
# Remove the last 10 rows for manual testing
fake_manual_testing = fakeNews.tail(10)
fakeNews = fakeNews.iloc[:-10, :]

In [45]:
true_manual_testing = trueNews.tail(10)
trueNews = trueNews.iloc[:-10, :]

In [46]:
# Combine datasets
merged_data = pd.concat([fakeNews, trueNews], axis=0)
merged_data.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [47]:
# Remove unwanted columns
dataframe = merged_data.drop(["title", "subject", "date"], axis=1)

In [48]:
# Check for null values
print(dataframe.isnull().sum())

text     0
class    0
dtype: int64


In [49]:
# Shuffle the dataset and reset the index
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
dataframe.head()

,text,class
0,Only supporters of Donald Trump could say some...,0
1,21st Century Wire says Trump supporters gather...,0
2,NEW YORK (Reuters) - New York Governor Andrew ...,1
3,WASHINGTON (Reuters) - U.S. President Donald T...,1
4,,0


In [50]:
#function to clean text
custom_stopwords = set(stopwords.words('english')) - {'not', 'no', 'because', 'since'}

def wordopt(text):
 # Lowercasing
    text = text.lower()

    # Removing special characters and punctuations
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)

    # Removing URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Removing HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Removing numbers
    text = re.sub(r'\w*\d\w*', '', text)

    # Expanding contractions
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'t", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'m", " am", text)

    # Tokenization
    tokens = word_tokenize(text)

    # Stopword removal
    tokens = [word for word in tokens if word not in custom_stopwords]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into text
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [51]:
dataframe['text']=dataframe['text'].apply(wordopt)

In [52]:
dataframe.head()

,text,class
0,supporter donald trump could say something stu...,0
1,century wire say trump supporter gathered rall...,0
2,new york reuters new york governor andrew cuom...,1
3,washington reuters u president donald trump te...,1
4,,0
